<a href="https://colab.research.google.com/github/Hrudayangam/amazon_hackathon/blob/main/Product_Browse_Node_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mounting Data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
from google.colab import drive
drive.mount('/content/drive')
!unzip -uq "https://drive.google.com/drive/u/2/folders/1HrUqrmw5iKoeAsIbxwTuD9xbeD3hckGC" -d "/content/drive/My Drive/amazon_hackathon"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
unzip:  cannot find or open https://drive.google.com/drive/u/2/folders/1HrUqrmw5iKoeAsIbxwTuD9xbeD3hckGC, https://drive.google.com/drive/u/2/folders/1HrUqrmw5iKoeAsIbxwTuD9xbeD3hckGC.zip or https://drive.google.com/drive/u/2/folders/1HrUqrmw5iKoeAsIbxwTuD9xbeD3hckGC.ZIP.


In [24]:
import pandas as pd
import csv

df = pd.read_csv("/content/drive/MyDrive/amazon_hackathon/check_train_data.csv", escapechar="\\", error_bad_lines=False, warn_bad_lines=False)

#using 30k data by anuj

# df = pd.read_csv(, quoting= csv.QUOTE_NONE)
# data = pd.read_csv("/content/drive/MyDrive/amazon_hackathon/check_train_data.csv", quoting= csv.QUOTE_NONE, escapechar="\\" )


In [25]:
df.head()

,Unnamed: 0,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4


In [26]:
pd.set_option("max_rows", 10)
df

,Unnamed: 0,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,2,The Ultimate Self-Sufficiency Handbook: A Comp...,NaN,Skyhorse Publishing,imusti,2
3,3,Amway Nutrilite Kids Chewable Iron Tablets (100),NaN,"[Nutrilite Kids,Chewable Iron Tablets,Quantity...",Amway,3
4,4,Teacher Planner Company A4 6 Lesson Academic T...,NaN,NaN,NaN,4
...,...,...,...,...,...,...
29991,29995,Fisher-Price Thomas The Train: Take-N-Play Spe...,Product Description Thomas and Friends Take-n-...,"[Sturdy die-cast construction,Magnets let you ...",Thomas & Friends,243
29992,29996,"Rise ""www.pannapaintings.com"" Digital Art Prin...","Our ""Exclusive"" series takes inspiration from ...","[Material: Print On Canvas,Size: 30 cm x 30 cm...",www.pannapaintings.com,700
29993,29997,Hose Nozzle ~ Solid Brass ~ Adjustable Spray P...,NaN,NaN,NaN,11244
29994,29998,Enlightened elephant by Babu Xavier Mousepad,Enlightened elephant by Babu Xavier,[A detailed painting of a mythological elephan...,Indian Colours,422


# Cleaning Data

In [46]:
nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.dropna(subset = ["TITLE"], inplace=True)
display(df.iloc[21736])

Unnamed: 0                                                    29998
TITLE                  Enlightened elephant by Babu Xavier Mousepad
DESCRIPTION                     Enlightened elephant by Babu Xavier
BULLET_POINTS     [A detailed painting of a mythological elephan...
BRAND                                                Indian Colours
BROWSE_NODE_ID                                                  422
Name: 29994, dtype: object

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21737 entries, 0 to 29994
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      21737 non-null  int64 
 1   TITLE           21737 non-null  object
 2   DESCRIPTION     21737 non-null  object
 3   BULLET_POINTS   21737 non-null  object
 4   BRAND           21667 non-null  object
 5   BROWSE_NODE_ID  21737 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 1.2+ MB


In [48]:
df.head(2)

,Unnamed: 0,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1


In [ ]:
# df[df.duplicated(['BROWSE_NODE_ID'], keep=False)]
# df.BROWSE_NODE_ID.duplicated()
# df.BROWSE_NODE_ID.duplicated().sum()
# df.loc[df.duplicated(), :]
# display(df.iloc[82])
# display(df.iloc[2902998])


In [49]:
# cleaning all the nan values in TITLE, DESCRIPTION, BULLET_POINTS

nan_value = float("NaN")
df.replace("", nan_value, inplace=True)
df.dropna(subset = ["TITLE"], inplace=True)
df.dropna(subset = ["DESCRIPTION"], inplace = True)
df.dropna(subset= ["BULLET_POINTS"], inplace = True)


# row_start = df.index[0]
# row_end = df.index[29994]
# df.loc[row_start:row_end, 'TITLE' : 'BRAND']
df

# dataset_ = df.loc[row_start:row_end, 'TITLE' : 'BRAND']
# dataset_

,Unnamed: 0,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
5,5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",Bhavya Enterprise,5
6,6,Glance Women's Wallet (Black) (LW-21),This Black wallet by Glance will be a treasure...,[The Most Comfortable Women's Wallet That You ...,Glance,6
7,7,Wild Animals Hungry Brain Educational Flash Ca...,Wild Animals are the animals that mostly stays...,[Playful learning: Flash cards develops the lo...,hungry brain,7
...,...,...,...,...,...,...
29987,29991,Deco Art Americana Decou Page Widemouth Paint ...,Americana Decou-Page Glue Widemouth Jar-8oz Na...,"[Best Quality,Deco Art,135962]",DecoArt,117
29988,29992,Invicta Pro-Diver Analog Gold Dial Men's Watch...,Missing short description,"[Dial Color: Gold, Case Shape: Round,Band Colo...",Invicta,289
29991,29995,Fisher-Price Thomas The Train: Take-N-Play Spe...,Product Description Thomas and Friends Take-n-...,"[Sturdy die-cast construction,Magnets let you ...",Thomas & Friends,243
29992,29996,"Rise ""www.pannapaintings.com"" Digital Art Prin...","Our ""Exclusive"" series takes inspiration from ...","[Material: Print On Canvas,Size: 30 cm x 30 cm...",www.pannapaintings.com,700


# BUNTY SENTIMENT ANALYSIS CODE

In [36]:


from pathlib import Path

import pandas as pd
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from google_drive_downloader import GoogleDriveDownloader as gdd
from torch.utils.data import DataLoader, Dataset
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm, tqdm_notebook



import nltk
#nltk.download()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random


In [37]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

DATA_PATH = '/content/drive/MyDrive/amazon_hackathon/check_train_data.csv'



In [58]:



class Sequences(Dataset):
    def __init__(self, path):
        df_ = pd.read_csv(path)
        self.vectorizer = CountVectorizer(stop_words='english', max_df=0.99, min_df=0.005)
        self.sequences = self.vectorizer.fit_transform(df.TITLE.tolist())
        self.sequences = self.vectorizer.fit_transform(df.DESCRIPTION.tolist())
        self.sequences = self.vectorizer.fit_transform(df.BULLET_POINTS.tolist())
        self.labels = df.BROWSE_NODE_ID.tolist()
        self.token2idx = self.vectorizer.vocabulary_
        self.idx2token = {idx: token for token, idx in self.token2idx.items()}
        
    def __getitem__(self, i):
        return self.sequences[i, :].toarray(), self.labels[i]
    
    def __len__(self):
        return self.sequences.shape[0]




In [80]:
dataset = Sequences(DATA_PATH)
train_loader = DataLoader(dataset, batch_size=4096)

print(dataset)

In [62]:

class BagOfWordsClassifier(nn.Module):
    def __init__(self, vocab_size, hidden1, hidden2):
        super(BagOfWordsClassifier, self).__init__()
        self.fc1 = nn.Linear(vocab_size, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, 1)
    
    def forward(self, inputs):
        x = F.relu(self.fc1(inputs.squeeze(1).float()))
        x = F.relu(self.fc2(x))
        return self.fc3(x)





In [61]:
model = BagOfWordsClassifier(len(dataset.token2idx), 128, 64)
model

BagOfWordsClassifier(
  (fc1): Linear(in_features=1387, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)

In [63]:

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=0.001)

model.train()

BagOfWordsClassifier(
  (fc1): Linear(in_features=1387, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)

In [64]:
train_losses = []
for epoch in range(5):
    progress_bar = tqdm_notebook(train_loader, leave=False)
    losses = []
    total = 0
    for inputs, target in progress_bar:
        model.zero_grad()

        output = model(inputs)
        loss = criterion(output.squeeze(), target.float())
        
        
        loss.backward()
              
        nn.utils.clip_grad_norm_(model.parameters(), 3)

        optimizer.step()
        
        progress_bar.set_description(f'Loss: {loss.item():.3f}')
        
        losses.append(loss.item())
        total += 1
     
    epoch_loss = sum(losses) / total
    train_losses.append(epoch_loss)
        
    tqdm.write(f'Epoch #{epoch + 1}\tTrain Loss: {epoch_loss:.3f}')
              



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch #1	Train Loss: -459.045


Epoch #2	Train Loss: -1912.509


Epoch #3	Train Loss: -4731.418


Epoch #4	Train Loss: -9322.788


Epoch #5	Train Loss: -15914.596


In [78]:

def predict_review(text):
    model.eval()
    with torch.no_grad():
        test_vector = torch.LongTensor(dataset.vectorizer.transform([text]).toarray())

        output = model(test_vector)
        prediction = torch.sigmoid(output).item()
        print(prediction)     
        # if prediction > 0.5:
        #     #print("the final result is:")
        #     return (' Positive Review')
        # else:
        #     #print("the final result is:")
        #     return(" Negative Review")

text='Wild Animals are the animals that mostly sta'
predict_review(text)

0.5881150364875793


# anuj


In [35]:


import seaborn as sns
import matplotlib.pyplot as plt 
from subprocess import check_output
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm 
from sklearn import metrics #checking model accuracy 
from sklearn.tree import DecisionTreeClassifier
import pandas as pd


train, test = train_test_split(df, test_size=None, train_size = 0.7) #splitting trainingdata in 70,30 ratio

##t_1 = train[train['sentiment']==1].sample(800,replace=True)
##t_2 = train[train['sentiment']==2].sample(800,replace=True)
##t_3 = train[train['sentiment']==0].sample(800,replace=True)

##training_bs = pd.concat([t_1, t_2, t_3])

print(train.shape)
##print(training_bs.shape)
print(test.shape)



(15215, 6)
(6522, 6)


In [ ]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

def Clean_data(df, column_name):  
  corpus = []
  try:
    for temp in df[column_name][:2]:
      review = re.sub('[^a-zA-Z0-9]', ' ', temp)
      review = review.lower()
      review = review.split()
      ps = PorterStemmer()
      all_stopwords = stopwords.words('english')
      all_stopwords.remove('not')
      review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
      review = ' '.join(review)
      corpus.append(review)


  except TypeError:
    print(temp)

  except KeyError:
    print(temp)
   # pass

  #print(len(corpus))
  return corpus

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
Clean_data(df,"TITLE")
len("TITLE")

5

In [ ]:
display(df.iloc[12685:12690])

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
12685,STAR MOON Ceramic Flower Vase with Retro Grid ...,NaN,[Top material: Made by the premium glazed ston...,STAR MOON,1711
12686,"Dolphin Miles Women Footwear Soft, Durable Sli...",Miles footwear presents stylish range of slipp...,[This brings you the best designer women's Fli...,Dolphin Miles,311
12688,Next Bazaar Micro Satin and Fibre You are The ...,<p><strong>VALENTINE DAY GIFT:</strong>&nbsp;A...,[Valentine Gifts For Girlfriend-Boyfriend Cush...,Next Bazaar,3547
12689,SM TELICOM® Soft Silicon [for Girls Boys] Prin...,Give a new style to your phone with this desig...,[Designer cover case. Beautiful and Tough. Use...,SM TELICOM,1045
12690,SNOWBALL Capri for Girls/Teens/Kids/Women Comb...,NaN,NaN,SNOWBALL,1190
